## 기존 불용어 사전에 추가하는 방식
  - 헤드라인은 명사 위주의 단어로 명사로 판단하기(품사선택은 추후 변경 가능)
  - 1글자 단어 제외(2글자 이상의 단어만 판단)
  
## 꼭 확인해야 되는 요소
- okt, mecab 등 한글 형태소 분석에 따라 추출되는 단어가 다르다는 점
- 자연어 처리 형태소 분석기 각각 독립적으로 판단해서 최종 결과치를 산출하고,
- okt와 mecab에서 공통으로 선정한 불용어 사전으로만 학습 진행한 최종 결과치를 또 okt, mecab을 써서 2번과 비교
- 가장 best 값을 나타낸 결과를 우리의 결과로 활용해볼 것


+ 추가 피드백 받은 것: 중립단어를 미리 제거하지 말고, 모델까지 다 돌려본 후 비교하는 과정으로 처리해볼것! 

## 최종 선정 전처리
- okt, mecab으로 형태소 분석 후 실질 형태소(뜻을 내포하는 핵심 형태소) 중심으로 판단하기에 mecab이 정확도와 속도에서 적절하다고 판단
- mecab으로 분리한 형태소에서 noise라고 판단하는 text는 삭제(3000여개)--> 부고, 인사 등 상승/하락에 영향이 없는 text
- ngram을 통해 시황이라는 내용이 상승/하락과 무관하다는 판단으로 시황이 들어간 wording 제거(8000여개)
- 

In [1]:
!sudo apt-get install -y fonts-nanum #코랩 한글 폰트 꺠짐 방지 폰트 설치
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (8,315 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 155229 files and d

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# 한국어 출력
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath,size=9)
plt.rc('font',family='NanumBarunGothic')


- train/test를 연도별로 index 기준으로 구분지을 예정이므로 date 기준으로 정렬한 후, index를 다시 매긴다. 

In [98]:
df = pd.read_csv('/content/drive/MyDrive/final_project/crawling_datasets.csv',index_col=0)
df = df.sort_values(by='date')
df.reset_index(inplace=True)
del df['index']
df

,name,headline,date,change
0,삼성전자,"[TV] 재계 총수들, 일자리 창출과 투자 확대 강조",2012-01-02,1
1,셀트리온,"[시황]상승동력 부재… 코스피, 하락 전환",2012-01-02,0
2,셀트리온,"코스닥, 새해 첫 거래일 소폭 오름세 지속",2012-01-02,0
3,셀트리온,코스닥 새해 첫 거래일 소폭 오름세 지속,2012-01-02,0
4,셀트리온,"[특징주]차바이오·알앤엘 등 바이오株, 연초 '강세'",2012-01-02,0
...,...,...,...,...
419439,SK하이닉스,"삼성전자만 11조원…연기금, 올해 24조원 팔았다",2021-12-30,1
419440,SK하이닉스,"SK하이닉스, 특별성과급 300% 쏜다",2021-12-30,1
419441,셀트리온,"삼성바이오로직스-셀트리온, 바이오 대장주 희비",2021-12-30,0
419442,LG화학,"3300도 뚫더니... '용두사미' 코스피, 마이너스 수익률 겨우 면했다",2021-12-30,0


In [100]:
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419444 entries, 0 to 419443
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   name      419444 non-null  object        
 1   headline  419444 non-null  object        
 2   date      419444 non-null  datetime64[ns]
 3   change    419444 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 12.8+ MB


In [101]:
# 중복으로 제거할 모든 rows의 index 확인 후 제거

temp = df.duplicated(subset='headline',keep=False)

df_duplicate = df.loc[temp,:]
erase_idx = df_duplicate.index.tolist()
erase_idx[:5], len(erase_idx)

([9, 10, 11, 12, 13], 174725)

In [102]:
df = df.drop(index=erase_idx)
df.reset_index(inplace=True)
del df['index']
df

,name,headline,date,change
0,삼성전자,"[TV] 재계 총수들, 일자리 창출과 투자 확대 강조",2012-01-02,1
1,셀트리온,"[시황]상승동력 부재… 코스피, 하락 전환",2012-01-02,0
2,셀트리온,"코스닥, 새해 첫 거래일 소폭 오름세 지속",2012-01-02,0
3,셀트리온,코스닥 새해 첫 거래일 소폭 오름세 지속,2012-01-02,0
4,셀트리온,"[특징주]차바이오·알앤엘 등 바이오株, 연초 '강세'",2012-01-02,0
...,...,...,...,...
244714,SK하이닉스,"""코로나 봉쇄로 삼성 시안공장 생산 줄여…메모리 공급사 주가에는 긍정적 영향""",2021-12-30,1
244715,SK하이닉스,"SK하이닉스, 인텔 낸드 1단계 인수",2021-12-30,1
244716,SK하이닉스,"SK하이닉스, 특별성과급 300% 쏜다",2021-12-30,1
244717,셀트리온,"삼성바이오로직스-셀트리온, 바이오 대장주 희비",2021-12-30,0


- 특정 종목의 상승 하락과 무관하다고 판단되는 헤드라인은 데이터에서 제외시킨다.
- 생톡관심종목,사진,포토, 부고,인사,부음 <표>,프로필, 오늘의 메모,집중관찰,<오늘의 화제주>,특징주,게임소개,업&다운 등 헤드라인 처음에 시작하는 단어 제외시키기


In [58]:
df_temp = df[~df.headline.str.startswith("[인사]")]
len(df),len(df_temp)

(244719, 244620)

In [103]:
useless = ["[생톡관심종목]",'[사진]','[포토]','[부고]','[인사]','[부음]','<표>','[프로필]','[오늘의 메모]','[집중관찰]','<오늘의 화제주>','[특징주]','[게임소개]','[업&다운]']
print("전체 데이터 개수 : ",len(df))
for eliminate_word in useless:
  df = df[~df['headline'].str.startswith(eliminate_word)]
print("남은 데이터 개수 ",len(df))

전체 데이터 개수 :  244719
남은 데이터 개수  241726


**전체 데이터에서 3000여개 정도 삭제된 것으로 확인됨**

In [108]:
# 시황 내용 포함시 제거 (상승/하락)
print('시황 키워드 행 제거 전 : ',len(df))
df = df[~df['headline'].str.contains('시황')]
print('시황 키워드 행 제거 후 : ',len(df))

시황 키워드 행 제거 전 :  241726
시황 키워드 행 제거 후 :  233704


**해당일의 코스피 가격, 시장을 중계해주는 시황 키워드는 상승/하락에 노이즈로 판단 기존 데이터에서 약 8000개 정도 삭제된 것으로 확인됨**

### 형태소 분석 전 전처리 기준
- 구두점
    - 구두점으로 표기된 형태는 공백으로 replace 처리하면서 1차 전처리 진행
  
- 한자
    - 헤드라인에 표기된 한자 자체만으로도 뜻이 있는 경우가 많았기에 자주 사용되는 한자로 표기되는 경우는 한글로 변경 작업 진행
    - ex) 국가명, 대통령 성씨, 기관(은행, 중소기업, 종목) 등이 포함됨
  
- 영어
    - 영어가 포함된 단어는 종목이름이 과반 이상이나, M&A, R&D, GDP 등 주요 경제 용어가 다수 포함되어 있어 띄어쓰기 기준으로라도 포함 진행
  
- 숫자
    - 숫자는 제외시킴
    - 경제 관련 기사에서 숫자 지표는 헤드라인 논조에서 중요한 역할을 차지할 수 있으나, 단위, 시간 등에 따라 다른 숫자에 대한 명확한 기준이 없기에 제외시킴
  
- 특수문자
    - ↓ , ↑ 와 같은 경우, 특수문자 자체로도 의미가 형성되므로 하락과 상승으로 변경 후 진행

- 기타
    - 헤드라인에 "미리보는"이라는 단어는 종목 상승,하락에 영향을 미치지 않는 것으로 판단되나 많은 양을 포함하므로 공백으로 제거 
  
**1차 전처리가 끝난 후, 한글, 영어, 띄어쓰기를 제외한 모든 표현 삭제(정규표현식 이용)** 

In [126]:
import re

def apply_regular_expression(headline):
  # 한글 추출 규칙 : 띄어쓰기(1 개)를 포함한 한글과 대문자, ... …는 반드시 공백으로 처리해야 띄어쓰기 유지
  headline = headline.replace("…"," ") # 구두점으로 표기하고 띄어쓰기가 반영이 되지 않은 것 처리
  headline = headline.replace(".."," ") # 구두점으로 표기하고 띄어쓰기가 반영이 되지 않은 것 처리
  headline = headline.replace("·"," ") # 구두점으로 표기하고 띄어쓰기가 반영이 되지 않은 것 처리

  # 국가나 기관등을 한자로 표기하는 경우가 많아 전처리 진행
  headline = headline.replace("中企","중소기업") # 중국과 혼동 피하기 위해 먼저 전처리(중국과 구분짓기 위함)
  headline = headline.replace("中","중국")
  headline = headline.replace("日","일본")
  headline = headline.replace("韓","한국")
  headline = headline.replace("美","미국")
  headline = headline.replace("銀","은행")
  headline = headline.replace('株','종목') # 많이 쓰이는 한자
  # 대동령을 한자로 표기하는 경우가 많아 전처리 진행
  headline = headline.replace('文','문재인')
  headline = headline.replace('朴','박근혜')

  headline = headline.replace("↓"," 하락")
  headline = headline.replace("↑", "상승")

  headline = headline.replace("미리보는","") # 미리보는 삭제
  headline = re.sub("\s\s+" , " ", headline) # 다중 공백 제거
 
  hangeul = re.compile('[^ ㄱ-ㅣ 가-힣a-zA-Z]') # 숫자는 배제!
  result = hangeul.sub('', headline)
  return result

In [83]:
apply_regular_expression('공모가 10% 프리미엄에도 청약경쟁률 663대 1···POSCO 관련종목 흥행 성공')

'공모가  프리미엄에도 청약경쟁률 대  POSCO 관련종목 흥행 성공'

In [ ]:
apply_regular_expression('中企 사회보험료 세액공제 확대')

'중소기업 사회보험료 세액공제 확대'

In [84]:
apply_regular_expression('sk하이닉스, OCI머티리얼즈 인수…반도체 역량 강화… 상승')

'sk하이닉스 OCI머티리얼즈 인수 반도체 역량 강화 상승'

- mecab, konlpy 설치

In [11]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 109 (delta 7), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (109/109), 1.27 MiB | 16.46 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [12]:
cd Mecab-ko-for-Google-Colab/

/content/Mecab-ko-for-Google-Colab


In [ ]:
!bash install_mecab-ko_on_colab_light_220111.sh

In [ ]:
!pip install konlpy

## mecab 분석 진행해보기

In [110]:
#한국어 형태소 분석 
from konlpy.tag import Okt,Mecab
from collections import Counter

okt = Okt()  # 형태소 추출 함수
mecab = Mecab()
print(okt.pos('sk하이닉스 OCI머티리얼즈 인수 반도체 역량 강화')) # 예시
print(mecab.pos('sk하이닉스 OCI머티리얼즈 인수 반도체 역량 강화'))

[('sk', 'Alpha'), ('하이닉스', 'Noun'), ('OCI', 'Alpha'), ('머티리얼즈', 'Noun'), ('인수', 'Noun'), ('반도체', 'Noun'), ('역량', 'Noun'), ('강화', 'Noun')]
[('sk', 'SL'), ('하이닉스', 'NNP'), ('OCI', 'SL'), ('머티', 'NNP'), ('리얼', 'NNG'), ('즈', 'XSN'), ('인수', 'NNG'), ('반도체', 'NNG'), ('역량', 'NNG'), ('강화', 'NNG')]


## mecab-ko-dic 품사 태그 (실질형태소 3대 용언)
- 체언 용언 수식언 외국어 (독립언(감탄사),형식형태소(관계언) 제외)
- 태그
  - 체언: ['NNG','NNP','NNB','NR','NP'] # 일반N,고유N,의존N(대다수 한글자라 제외가능성 큼), 수사 대명사 
  - 용언:['VV','VA','VCP','VCN'] # 동사 형용사 긍정지정사 부정지정사
  - 수식언: ['MM','MAG'] : #관형사, 일반부사
  - 외국어 : 영어만 포함시킴 'SL'

In [86]:
mecab.pos('SK공모가 프리미엄에도 청약경쟁률 대 흥행 성공')

[('SK', 'SL'),
 ('공모', 'NNG'),
 ('가', 'JKS'),
 ('프리미엄', 'NNG'),
 ('에', 'JKB'),
 ('도', 'JX'),
 ('청약', 'NNG'),
 ('경쟁', 'NNG'),
 ('률', 'XSN'),
 ('대', 'XPN'),
 ('흥행', 'NNG'),
 ('성공', 'NNG')]

In [85]:
okt.pos('SK공모가 프리미엄에도 청약경쟁률 대 흥행 성공')

[('SK', 'Alpha'),
 ('공모', 'Noun'),
 ('가', 'Josa'),
 ('프리미엄', 'Noun'),
 ('에도', 'Josa'),
 ('청약', 'Noun'),
 ('경쟁률', 'Noun'),
 ('대', 'Verb'),
 ('흥행', 'Noun'),
 ('성공', 'Noun')]

- okt와 비교할 때 더 상세한 형태소 분석가능(mecab: 세종 품사 태그 기준)

In [111]:
consider_pos = ['NNG','NNP','NNB','NR','NP','VV','VA','VCP','VCN','MM','MAG','SL']
def mecab_words(headline):
  words = []
  for word, pos in mecab.pos(headline):
      if pos in consider_pos:
        words.append(word)
  return words

In [112]:
from tqdm import tqdm

def headline_keywords(raw):
  return mecab_words(apply_regular_expression(raw))

token_list = []
for idx,row in tqdm(df.iterrows()):
  raw = row['headline']
  per_words = headline_keywords(raw)
  per_words = [x for x in per_words if len(x) > 1]  # 한글자 키워드 제거
  corpus = ' '.join(per_words)
  token_list.append(corpus)

233704it [00:41, 5599.51it/s]


In [113]:
df['token_mecab'] = token_list
df.tail(20)

,name,headline,date,change,token_mecab
244698,LG화학,"개미 울리는 물적분할에 고승범 ""법적 측면 포함 대책 검토""",2021-12-30,0,개미 울리 물적분할 고승범 측면 포함 대책 검토
244699,LG화학,올해 증시 마지막 날 코스피 2970선으로 마감...한 해 동안 3.63% 상승,2021-12-30,0,올해 증시 마지막 코스피 마감 동안 상승
244700,LG화학,"코스피 마지막 거래일 2,970선 마감‥삼성전자 이틀째 하락 흐름",2021-12-30,0,코스피 마지막 거래일 마감 삼성전자 이틀 하락 흐름
244701,LG화학,3000선 끝내 못 지킨 코스피…올해 2977.65 마감,2021-12-30,0,끝내 코스피 올해 마감
244702,LG화학,"JW중외제약, 아토피신약 개발 박차…유한양행·LG화학도 경쟁",2021-12-30,0,JW 중외 제약 아토피 신약 개발 박차 유한양행 LG 화학 경쟁
244704,LG화학,"악재 덮친 LG화학, 주가 뚝뚝…""50만원대 추락"" 최악 시나리오까지",2021-12-30,0,악재 LG 화학 뚝뚝 추락 최악 시나리오
244705,LG화학,"금융위원장 “물적분할 규제 검토, 인터넷은행 대출 상한도 업권 특성 고려할 것”",2021-12-30,0,금융 위원장 물적분할 규제 검토 인터넷 은행 대출 상한 특성 고려
244706,카카오,올해 코스피 시총 2200조원 돌파로 마감,2021-12-30,0,올해 코스피 돌파 마감
244707,카카오,[2021 증시결산] 코스피 3.6% 올라 G20 18위…시총 2천200조원 넘어,2021-12-30,0,증시 결산 코스피
244708,SK하이닉스,"""최태원이 쏜다""···SK하이닉스, 전 직원 특별 성과급 300% 지급",2021-12-30,1,최태원 SK 하이닉스 직원 특별 성과급 지급


In [127]:
df.reset_index(inplace=True)
del df['index']

In [128]:
df.to_csv('./final_preprocessing_df',encoding='utf-8')

---